In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_dos(nx, ny, hz, V, nk=50, nbins=100):
    """
    Compute density of states using histogram method.
    
    Parameters
    ----------
    nx, ny : int
        Plane-wave cutoffs along reciprocal lattice directions.
    hz : float
        magnetization parameter.
    V : float
        Lattice potential amplitude.
    nk : int
        Number of k-points along each direction in the Brillouin zone.
    nbins : int
        Number of bins for the histogram.
    
    Returns
    -------
    energy_bins : ndarray
        Energy bin centers.
    dos : ndarray
        Density of states.
    """
    
    # Reciprocal lattice vectors for triangular lattice
    a = 1
    ax = a
    ay = np.sqrt(3) * a
    
    b1 = np.array([2*np.pi/ax, 0])
    b2 = np.array([2*np.pi/ax * (-1/2), 2*np.pi/ay * (1/2)])
    
    # First Brillouin zone vertices (hexagon)
    # For triangular lattice, the BZ is a hexagon
    # We'll sample the parallelogram defined by b1 and b2, 
    # which tiles the BZ properly
    
    # Collect all eigenvalues
    all_eigenvalues = []
    k_points = []
    
    print(f"Computing eigenvalues for triangular lattice BZ...")
    
    count = 0
    # Sample the first BZ as a parallelogram
    for i in range(nk):
        for j in range(nk):
            # Map to [0,1] x [0,1] and then to BZ
            u = i / nk
            v = j / nk
            
            # k-point in the first BZ
            k = u * b1 + v * b2
            kx, ky = k[0], k[1]
            
            k_points.append([kx, ky])
            
            H = build_hamiltonian(nx, ny, kx, ky, hz, V)
            eigenvalues = np.linalg.eigvalsh(H)
            all_eigenvalues.extend(eigenvalues)
            count += 1
    
    all_eigenvalues = np.array(all_eigenvalues)
    k_points = np.array(k_points)
    
    print(f"Total k-points sampled: {count}")
    
    # Create histogram
    dos, bin_edges = np.histogram(all_eigenvalues, bins=nbins, density=True)
    energy_bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    return energy_bins, dos, all_eigenvalues


def plot_dos(energy_bins, dos, all_eigenvalues=None):
    """
    Plot the density of states.
    
    Parameters
    ----------
    energy_bins : ndarray
        Energy bin centers.
    dos : ndarray
        Density of states.
    all_eigenvalues : ndarray, optional
        All eigenvalues for reference plot.
    """
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot DOS
    axes[0].plot(energy_bins, dos, 'b-', linewidth=2)
    axes[0].fill_between(energy_bins, dos, alpha=0.3)
    axes[0].set_xlabel('Energy', fontsize=12)
    axes[0].set_ylabel('Density of States', fontsize=12)
    axes[0].set_title('Density of States (C6 Hamiltonian)', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    # Plot histogram of eigenvalues
    if all_eigenvalues is not None:
        axes[1].hist(all_eigenvalues, bins=100, alpha=0.7, edgecolor='black')
        axes[1].set_xlabel('Energy', fontsize=12)
        axes[1].set_ylabel('Count', fontsize=12)
        axes[1].set_title('Eigenvalue Distribution', fontsize=14)
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
